In [150]:
from seabreeze.spectrometers import Spectrometer
from matplotlib import pyplot as plt
import time
import numpy as np
import serial
import sys
import numpy as np
import csv
from scan_functions import *
import pandas as pd
import sqlite3
import datetime
from scipy.signal import savgol_filter as sgf
from math import factorial as fact
from tkinter import Tk, filedialog
from scipy.signal import find_peaks as peaks
from IPython.display import clear_output
# spec = Spectrometer.from_first_available()

time_in_ms = int(input("Set integration time (in ms): "))*10**3
# spec.integration_time_micros(time_in_ms) # Sets integration time
port = input("Which port number is is the teensy?: ").strip()
conn = sqlite3.connect('Database/Sniffing-Sensor.db') # connect to the database
c = conn.cursor() # Creates a cursor to interact with database
# ser = serial.Serial("COM{}".format(port), 9600) # Opening serial port to communicate with teensy
output_name = input("What do you want to call the output file?: ").strip()

Set integration time (in ms): 10
Which port number is is the teensy?: 4
What do you want to call the output file?: test1


In [ ]:
root = Tk()
root.fileName = filedialog.askopenfilename( filetypes = ( ("Excel File", "*.xlsx"), ("All Files", "*.*") ) )

experiment = pd.read_excel('{}'.format(root.fileName))
experiment.set_index("True Runs", inplace=True) # indexed by the True run number
experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows

# Set output file
to_df = {"Run": [], 
         "Steady State": [], 
         "Peak 1": [], 
         "Peak 2": [], 
         "Peak 3": [], 
         "Trough 1": [], 
         "Trough 2": []}

df2 = pd.DataFrame(to_df)
df2.set_index('Run', inplace=True)
df2.to_excel('../Experiment Files/{}.xlsx'.format(output_name))

num_exp = len(experiment.index)
run = 0

while run < num_exp: 
    clear_output() # clears output after every run
    
    # --------------- DESIGNING RECIPE -------------------
    
    o_time = list(experiment['Odorant Time'])[run]
    odorant_time = [o_time for i in range(3)]
    
    purge = list(experiment['Purge time'])[run]
    
    c_time = list(experiment['Clean Time'])[run]
    clean_time = [purge] + [c_time for i in range(2)]
    
    delay_time = 10 # scans per second

    
    # --------------- PERFORMING RECIPE -------------------
    
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1

    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    # Compound name coded into its OdorID from the database
    compound = list(experiment['Current'])[run]
    temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound))
    OdorID = c.fetchall()[0][0]
    print("Experiment Name: {}\nCompound: {}".format(exp_name, compound))


    # Procedure name coded into its ProcedureID from the database
    temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
    ProcedureID = c.fetchall()[0][0]

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/CSV/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    df = pd.DataFrame()
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")

    # ============================== RECIPE ==============================

    intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse = [0] #empty list to be populated with time values

    compound_dict = {"Nitrogen": Nitrogen, 
                     "Water": Water, 
                     "IPA": IPA, 
                     "Ethanol": Ethanol} # Assigns the compound to its injection function

    # If there is a nitrogen flush specified beforehand, then do a 10 minute flush.
    if list(experiment['Preceding'])[run] == "Nitrogen":
        print("Purging chamber for 10 minutes. Please standby. ")
        nitro_arg = [[], ser, intensities, spec, elapse, True]
        Nitrogen(*nitro_arg)
    
    for i in range(len(odorant_time)): 
        nitro_arg = [clean_time[i], ser, intensities, spec, elapse]
        Nitrogen(*nitro_arg)

        odorant_arg = [odorant_time[i], ser, intensities, spec, delay_time, elapse]
        compound_dict["{}".format(compound)](*odorant_arg) # Change the function based on what odorant you are testing

    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

    sys.stdout.write("\n{}\nDone Captures!\n".format('#'*30))

    # ========================== WRITING TO FILES ==========================
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

    plot_river(np.array(intensities),plt, np) # Plotting river plot

    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file


    Notes = "Testing automation."
    Notes += "\nOdorant Times: {}\nClean Times: {}\n".format(", ".join([str(i) for i in clean_time]),
                                                             ", ".join([str(i) for i in odorant_time]))

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQL

    # ========================== DATA ANALYSIS ==========================

    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)
    
    
    # ========================== RESPONSE ACQUISITION ==========================
    
    ## STEADY STATE
    
    def steady_state(seq, n=60, eps=0.2):
        seq = abs(seq) < eps
        for i in range(len(seq)):
            win = seq[i:i+n]
            if sum(win) == n:
                return int(i+n/2), i
        return 0

    ss_pd = phase_deriv[:purge-20] # Use only the part of the spectrum when nitrogen is injected
    
    # Use half of one standard deviation as the steady state height condition (only applies to purge time)
    eps = min([np.mean(ss_pd) + np.std(ss_pd), np.mean(ss_pd) + np.std(ss_pd)])/2

    threshold = steady_state(ss_pd, eps=eps)[0]
    threshold_i = steady_state(ss_pd, eps=eps)[1]

    ## PEAKS AND TROUGHS
    ss_pd2 = phase_deriv[purge-20:] # Use only up until first odorant is injected
    eps2 = max([np.mean(ss_pd2) + np.std(ss_pd2), np.mean(ss_pd2) + np.std(ss_pd2)]) # take standard deviation of odorant part

    peak_list = [purge-20+i for i in list(peaks(ss_pd2, eps2))[0]]
    peak_height_vals = [i for i in peaks(ss_pd2, eps2)[1]['peak_heights']] # defining peak heights
    peak_height = [i for i in peak_height_vals if i in sorted(peak_height_vals, reverse=True)[:3]] # reverse to start with largest values
    peak_indices = [(i,j) for i,j in enumerate(peak_height_vals)]
    peak_indices.sort(key=lambda x: x[1])
    peak_list = [peak_list[i] for i in range(len(peak_list)) if i in [i[0] for i in peak_indices]][:3] # only take the highest 3 values

    trough_list = [purge-20+i for i in list(peaks(-ss_pd2, eps2))[0]]
    trough_height_vals = [i for i in peaks(-ss_pd2, eps2)[1]['peak_heights']]
    trough_height = [i for i in trough_height_vals if i in sorted(trough_height_vals, reverse=True)[:2]]
    trough_indices = [(i,j) for i,j in enumerate(trough_height_vals)]
    trough_indices.sort(key=lambda x:x[1], reverse=True)
    trough_list = [trough_list[i] for i in range(len(trough_list)) if i in [i[0] for i in trough_indices][:2]]
    
    fig = plt.figure(figsize=(14, 7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), 
                           xlabel="time (s)", 
                           ylabel="Absolute Difference")

    dfcols = [float(i) for i in df.columns[:-1]] # we lose a single value by taking derivative of phase
    pder.plot(dfcols, phase_deriv)
    for i in peak_list: 
        pder.plot(float(df.columns[i]), phase_deriv[i], 'r*')
    for i in trough_list:
        pder.plot(float(df.columns[i]), phase_deriv[i], 'g*')
    pder.plot(float(df.columns[threshold]), phase_deriv[threshold], 'k*')

    plt.axvspan(float(df.columns[threshold-30]),float(df.columns[threshold+30]), color="y", alpha=0.2) # plot steady state window
    plt.axhspan(-eps,eps, color="b", alpha=0.1) # plot steady state std window
    plt.axhspan(-eps2,eps2, color="b", alpha=0.1) # plot peak std window
    plt.vlines(float(df.columns[threshold]),min(phase_deriv), max(phase_deriv), alpha=0.5, linestyles={'dashed'}, label='{}'
               .format((float(df.columns[threshold]), round(phase_deriv[threshold],2))))
    plt.legend()

    plt.savefig("../Analysis Images/{}_SS_Peaks.png".format(exp_name))
    plt.close(fig)
    
    # -------------------------- SAVE RESPONSE VARIABLES TO SPREADSHEET -------------------------------
    to_df['Run'].append(run)
    to_df['Steady State'].append(threshold)
    to_df['Peak 1'].append(peak_height[0])
    to_df['Peak 2'].append(peak_height[1])
    to_df['Peak 3'].append(peak_height[2])
    to_df['Trough 1'].append(trough_height[0])
    to_df['Trough 2'].append(trough_height[1])
    
    df2 = pd.DataFrame(to_df)
    df2.set_index('Run', inplace=True)
    df2.head()
    df2.to_excel('../Experiment Files/{}.xlsx'.format(output_name))
    
ser.close() # Close serial connection
c.close() # Close cursor 
conn.close() # Close connection to save memory

### TESTING PLOTS AND SAVING DATA 

In [151]:
# Set output file
to_df = {"Run": [], 
         "Steady State": [], 
         "Peak 1": [], 
         "Peak 2": [], 
         "Peak 3": [], 
         "Trough 1": [], 
         "Trough 2": []}

df2 = pd.DataFrame(to_df)
df2.set_index('Run', inplace=True)
df2.to_excel('../Experiment Files/test1.xlsx')
purge = 600

for run in [10,11,12,13,14]:
    clear_output()
    # variables 
    exp_name = run
    compound = "Test"

    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)


    ## FOURIER TRANSFORM
    print("Fourier transform of {}".format(run))
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Test/{}_fourier.png".format(exp_name))
    plt.close(fig)

    ## PHASE CALCULATION
    print("Phase of {}".format(run))
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Test/{}_phase.png".format(exp_name))
    plt.close(fig)

    ## PHASE DERIVATIVE CALCULATION
    print("Phase of {}".format(run))
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Test/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)


    # ========================== RESPONSE ACQUISITION ==========================
    print("Populating response values of {}".format(run))
    ## STEADY STATE

    def steady_state(seq, n=60, eps=0.2):
        seq = abs(seq) < eps
        for i in range(len(seq)):
            win = seq[i:i+n]
            if sum(win) == n:
                return int(i+n/2), i
        return 0

    ss_pd = phase_deriv[:purge-20] # Use only the part of the spectrum when nitrogen is injected

    # Use half of one standard deviation as the steady state height condition (only applies to purge time)
    eps = min([np.mean(ss_pd) + np.std(ss_pd), np.mean(ss_pd) + np.std(ss_pd)])/2

    threshold = steady_state(ss_pd, eps=eps)[0]
    threshold_i = steady_state(ss_pd, eps=eps)[1]

    ## PEAKS AND TROUGHS
    ss_pd2 = phase_deriv[purge-20:] # Use only up until first odorant is injected
    eps2 = max([np.mean(ss_pd2) + np.std(ss_pd2), np.mean(ss_pd2) + np.std(ss_pd2)]) # take standard deviation of odorant part

    peak_list = [purge-20+i for i in list(peaks(ss_pd2, eps2))[0]]
    peak_height_vals = [i for i in peaks(ss_pd2, eps2)[1]['peak_heights']] # defining peak heights
    peak_height = [i for i in peak_height_vals if i in sorted(peak_height_vals, reverse=True)[:3]] # reverse to start with largest values
    peak_indices = [(i,j) for i,j in enumerate(peak_height_vals)]
    peak_indices.sort(key=lambda x: x[1])
    peak_list = [peak_list[i] for i in range(len(peak_list)) if i in [i[0] for i in peak_indices]][:3] # only take the highest 3 values

    trough_list = [purge-20+i for i in list(peaks(-ss_pd2, eps2))[0]]
    trough_height_vals = [i for i in peaks(-ss_pd2, eps2)[1]['peak_heights']]
    trough_height = [i for i in trough_height_vals if i in sorted(trough_height_vals, reverse=True)[:2]]
    trough_indices = [(i,j) for i,j in enumerate(trough_height_vals)]
    trough_indices.sort(key=lambda x:x[1], reverse=True)
    trough_list = [trough_list[i] for i in range(len(trough_list)) if i in [i[0] for i in trough_indices][:2]]

    fig = plt.figure(figsize=(14, 7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), 
                           xlabel="time (s)", 
                           ylabel="Absolute Difference")

    dfcols = [float(i) for i in df.columns[:-1]] # we lose a single value by taking derivative of phase
    pder.plot(dfcols, phase_deriv)
    for i in peak_list: 
        pder.plot(float(df.columns[i]), phase_deriv[i], 'r*')
    for i in trough_list:
        pder.plot(float(df.columns[i]), phase_deriv[i], 'g*')
    pder.plot(float(df.columns[threshold]), phase_deriv[threshold], 'k*')

    plt.axvspan(float(df.columns[threshold-30]),float(df.columns[threshold+30]), color="y", alpha=0.2) # plot steady state window
    plt.axhspan(-eps,eps, color="b", alpha=0.1) # plot steady state std window
    plt.axhspan(-eps2,eps2, color="b", alpha=0.1) # plot peak std window
    plt.vlines(float(df.columns[threshold]),min(phase_deriv), max(phase_deriv), alpha=0.5, linestyles={'dashed'}, label='{}'
               .format((float(df.columns[threshold]), round(phase_deriv[threshold],2))))
    plt.legend()

    plt.savefig("../Test/{}_SS_Peaks.png".format(exp_name))
    plt.close(fig)

    # -------------------------- SAVE RESPONSE VARIABLES TO SPREADSHEET -------------------------------
    print("Run number {}".format(run))
    to_df['Run'].append(i)
    to_df['Steady State'].append(threshold)
    to_df['Peak 1'].append(peak_height[0])
    to_df['Peak 2'].append(peak_height[1])
    to_df['Peak 3'].append(peak_height[2])
    to_df['Trough 1'].append(trough_height[0])
    to_df['Trough 2'].append(trough_height[1])

    df2 = pd.DataFrame(to_df)
    df2.set_index('Run', inplace=True)
    df2.head()
    df2.to_excel('../Experiment Files/test1.xlsx'.format(output_name))
    
print("Done!")


Fourier transform of 14
Phase of 14
Phase of 14
Populating response values of 14
Run number 14
Done!
